In [60]:
# Get some libs in your life
import sys, os, time
import geopandas as gpd
import pandas as pd
import rasterio as rt
import numpy as np
from affine import Affine
from shapely.geometry import Point, LineString
from functools import partial
import pyproj
from shapely.ops import transform
from shapely.geometry import box
from shapely.wkt import loads
from rasterio import features
import networkx as nx

In [2]:
# set path to the DEM model
pth = r'C:\Users\charl\Documents\GOST\Yemen\SRTM\clipped'
f = r'clipped_e20N40.tif'

# import DEM model using rasterio
DEM_obj = rt.open(os.path.join(pth, f))

# set arr as the array of raster vals
arr = DEM_obj.read()
arr = arr[0]

### Import Population raster as base, sample the DEM

In [10]:
bpth = r'C:\Users\charl\Documents\GOST\Yemen\graphtool'
f = r'origins_1km_snapped.csv'
df = pd.read_csv(os.path.join(bpth, f))

In [15]:
list_of_nodes = {}
for index, data in df.iterrows():
    list_of_nodes.update({index:(data['Longitude'], data['Latitude'])})

dataset = DEM_obj

# create list of values, throw out nodes that don't intersect the bounds of the raster
b = dataset.bounds
datasetBoundary = box(b[0], b[1], b[2], b[3])
selKeys = []
selPts = []
for key, pt in list_of_nodes.items():
    if Point(pt[0], pt[1]).intersects(datasetBoundary):
        selPts.append(pt)
        selKeys.append(key)
raster_values = list(dataset.sample(selPts))
raster_values = [x[0] for x in raster_values]

# generate new dictionary of {node ID: raster values}
ref = dict(zip(selKeys, raster_values))

In [30]:
DEM_VALS = pd.DataFrame({'DEM_Value':list(ref.values())}, index = ref.keys())
df['DEM_Value'] = DEM_VALS.DEM_Value
df.loc[df.DEM_Value < 0, 'DEM_Value'] = 0
df = gpd.GeoDataFrame(df, geometry = df['geometry'].apply(loads), crs = {'init':'epsg:4326'})

### Burn these DEM values back to the original Population raster on another band

In [53]:
rst_fn = os.path.join(bpth,'pop18_resampled.tif')
out_fn = os.path.join(bpth,'pop18_resampled_DEM.tif')

# Update metadata
rst = rt.open(rst_fn, 'r')
meta = rst.meta.copy()
D_type = rt.float64
meta.update(compress='lzw', dtype = D_type, count = 2)

with rt.open(out_fn, 'w', **meta) as out:
    with rt.open(rst_fn, 'r') as pop:
        
        # this is where we create a generator of geom, value pairs to use in rasterizing
        shapes = ((geom,value) for geom, value in zip(df.geometry, df.DEM_Value))

        population = pop.read(1).astype(D_type)
        cpy = population.copy()

        DEM_VALS = features.rasterize(shapes=shapes, fill=0, out=cpy, transform=out.transform)

        out.write_band(1, population)
        out.write_band(2, DEM_VALS)

### Take new Pop / Dem hybrid, make walking graph out of it

In [57]:
pth = bpth
f = r'pop18_resampled_DEM.tif'

# import DEM model using rasterio
DEM_obj = rt.open(os.path.join(pth, f))

# set arr as the array of raster vals
arr = DEM_obj.read()
arr = arr[1]

In [58]:
# Tobler's hiking function: https://en.wikipedia.org/wiki/Tobler%27s_hiking_function
def speed(incline_ratio, max_speed):
    walkspeed = max_speed * np.exp(-3.5 * abs(incline_ratio + 0.05)) 
    return walkspeed

In [61]:
# write function to get geometry of centroid of raster cell from array position using Affine transformation
T0 = DEM_obj.transform
T1 = T0 * Affine.translation(0.5, 0.5)
rc2xy = lambda r, c: (c, r) * T1

# Set projections
source = 'epsg:4326'
target = 'epsg:32638'

# set function for reprojecting geometry from source to target
project_WGS_UTM = partial(
            pyproj.transform,
            pyproj.Proj(init=source),
            pyproj.Proj(init=target))

# Set walking speed calculation parameters
max_walkspeed = 5
max_ratio = 1.3

# open some blank variables
node_list,edge_list = [],[]
edge_counter = 0

# Iterate through raster array, generate Graph. For every raster cell (bar the outer edges):
for x in range(1, (arr.shape[0] - 1)):
    for y in range(1, (arr.shape[1] - 1)):
        
        # current elev
        val = arr[x][y]
        
        if val != DEM_obj.nodata:
            
            # node ID ('u') is the position in the array
            u = (x, y)
            
            # get the geometry of that centroid
            point_geom = Point(rc2xy(x,y))
            
            # fill in the data dictionary for the new node
            data = {'Elevation': val,
                   'geometry':point_geom,
                    'x':point_geom.x,
                    'y':point_geom.y}
            
            # add completed node tuple (u, data) to the node list
            node_list.append((u, data))
            
            # define a range of adjustments - to reach the 8 cells adjacent to the current cell
            for x_adj in [-1,0,1]:
                for y_adj in [-1,0,1]:
                    
                    # ignore the 0,0 adjustment combo - that's current cell!
                    if x_adj == 0  and y_adj == 0:
                        pass
                    
                    # define an offset x and y
                    else:
                        xD = (x+x_adj)
                        yD = (y+y_adj)
                        
                        # label of new destination node - v - is the adjusted cell position. 
                        # you can only get away with this whilst u = (x,y). 
                        v = (xD,yD)
                        
                        # elevation of target cell is just the array value at that address
                        valD = arr[xD][yD]
                        
                        # as long as that's not garbage...
                        if valD != DEM_obj.nodata:
                            
                            # geometry of this edge (only needed for visualizing...) is a line between the two
                            # cell centroids. Use our rc2xy function to pick up both points, make a line between them
                            geometry = LineString([Point(rc2xy(x,y)),Point(rc2xy(x+x_adj,y+y_adj))])
                            
                            # reproj this geometry to find the distance in meters between them
                            projected_geometry = transform(project_WGS_UTM, geometry)
                            distance_metres = projected_geometry.length
                            
                            # calculate the elevation delta                            
                            delta_elevation = (valD - val)
                            
                            # calculate vertical delta / horizontal delta
                            incline_ratio = delta_elevation / distance_metres 
                            
                            # Use Tobler's walking speed function, within a range of inclines
                            if -max_ratio < incline_ratio < max_ratio:
                                speed_kmph = speed(incline_ratio = incline_ratio, max_speed = max_walkspeed)
                            
                            # if too steep for Tobler, set a seriously slow traverse speed
                            else:
                                speed_kmph = 0.02
                            
                            # assemble new edge dictionary
                            data = {'walkspeed_kmph':speed_kmph,
                                   'geometry':geometry,
                                   'length': distance_metres,
                                   'traverse_time': ((distance_metres / 1000) / speed_kmph),
                                   'ID':edge_counter}
                            
                            # add to edge list
                            edge_list.append((u, v, data))
                            
                            # iterate up the edge ID for each new edge generated
                            edge_counter+=1

# gimme a blank graph, add nodes, add edges. 
G = nx.MultiDiGraph()
G.add_nodes_from(node_list)
G.add_edges_from(edge_list)

print('Finished. New graph has %s nodes and %s edges' % (G.number_of_nodes(), G.number_of_edges()))

Finished. New graph has 549095 nodes and 4378588 edges


In [62]:
# save the graph down for use later
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets')
import GOSTnet as gn
gn.save(G, 'trial', r'C:\Users\charl\Documents\GOST\Yemen\walk_graph')

peartree version: 0.6.0 
networkx version: 2.2 
matplotlib version: 2.2.2 
osmnx version: 0.8.2 


### Testing the walkgraph

In [165]:
# test the graph. Make a shapefile with a couple of points, calc the travel time + shortest paths between them
test_shp = gpd.read_file(os.path.join(r'C:\Users\charl\Documents\GOST\Yemen\SRTM','test.shp'))

# snap our test shapefile to our new graph
snapped_test = gn.pandana_snap(G, test_shp)

# define a single nodes as origin, destinations
origin = snapped_test.NN.iloc[1]
destination = snapped_test.NN.iloc[4]

# do some shortest path stuff
t_path = nx.shortest_path(G, origin, destination, weight = 'traverse_time')
t_time = nx.shortest_path_length(G, origin, destination, weight = 'traverse_time')

# now, go back through the graph, find the edges that are in the t_path list of shortest edges. 
IDs = []
for u, v, data in G.edges(data = True):
    if u in t_path:
        if v in t_path:
            IDs.append(data['ID'])

# save that down as a .csv you can visualize. 
edge_gdf = gn.edge_gdf_from_graph(G)
edge_gdf_slice = edge_gdf.loc[edge_gdf.ID.isin(IDs)]
edge_gdf_slice.to_csv(os.path.join(r'C:\Users\charl\Documents\GOST\Yemen\SRTM','test_path.csv'))

((0, 1219), {})
((0, 1220), {})
((0, 1221), {})
((414, 0), {})
((774, 1525), {})
((826, 1376), {})
((826, 1377), {})
